## Sample Quiz

ADNOC is conducting oil reserve exploration across UAE. They predict whether oil is present in a given location using a sophisticated device which works in the following way: once the device is activated, it returns a signal which is a real number.

Studies have found that when oil is not present, the signal follows Laplace distribution with mean $-1$, and scale $1/2$; that is, if $X$ denotes the random signal, then $X$ follows the density
$$
f_0(x) = p(x|\text{no oil}) = \exp\left(-\frac{|x+1|}{2}\right).
$$
When the location has oil, the density of the signal follows Laplace distribution with mean $5$ and scale $1/2$, i.e.
$$
f_1(x) = p(x|\text{oil}) = \exp\left(-\frac{|x-5|}{2}\right).
$$



### Problem 1

In [1]:
#Run this cell to import numpy
import numpy as np

**Task 1 [0.5 point]:** Implement the functions $f_0$ and $f_1$. 

In [2]:
def f_0(x):
  #Return f_0(x)
  return np.exp(- np.abs(x + 1) / 2)

def f_1(x):
  #Return f_1(x)
  return np.exp(- np.abs(x - 5) / 2)

Our goal in this part is implementing the *optimal binary predictor* for whether the drilling at a given location is viable.


Each drilling incurs a cost of  2  million Dirhams. If after the drilling the oil is found, the company earns  102  million Dirhams. 

Thus, if we denote the absence of oil with $0$ and presence of oil with $1$, following the notation from the course we have
\begin{align*}
&loss(0, 0) = 0,\\ &loss(0, 1) = 0, \\ &loss(1, 0) = 2 \cdot 10^6,\\ &loss(1, 1) = -10^8.
\end{align*}


Previous experience has shown that the probability of oil presence *prior to any measurement* is equal to $0.1$, so $p_0 = 0.9$, and $p_1 = 0.1$. 

**Task 2 [0.5 point]:** Implement the function which computes 
$$
\log\mathcal{L}(x) = \log\left(\frac{f_1(x)}{f_0(x)}\right).
$$


In [3]:
def log_L(x):
  #Return log L(x)
  return -np.abs(x - 5) / 2 + np.abs(x + 1) / 2

**Task 3 [0.5 point]:** By equation $(12)$ from slide $103$, the optimal binary predictor outputs $1$ if and only if 
$$
\log \mathcal{L}(x) \geq \log\left(\frac{p_0(loss(1, 0) - loss(0,0))}{p_1(loss(0,1) - loss(1, 1))}\right).
$$

Using this, implement the function drilling() which implements the optimal binary predictor for the problem in question

In [4]:
def drilling(x):
  return (log_L(x) >= np.log(9) + np.log(2 / 10 ** 2)).astype('int')

**Task 4 [0.5 point]:** Test the function drilling for $x = -1,\text{ } 0,\text{ } 1$.

In [5]:
for i in [-1, 0, 1]:
  print(drilling(i))

0
0
1


### Problem 2

In this part, we will implement the Perceptron algorithm with the data as generated below. 

In [6]:
rng = np.random.default_rng(12345)

n = 100
d = 5
X = rng.standard_normal((n, d))
Y = rng.binomial(1, 1/2, n)
Y = 2 * Y - 1

X = np.concatenate((X, Y.reshape(-1, 1)), axis=1)
Q, _ = np.linalg.qr(rng.standard_normal((d + 1, d + 1)))
X = X @ Q

# Try to think why are this data separable?

**Task 5 [1 point]:** Implement the Perceptron algorithm by filling in the missing parts.

In [7]:
# Perceptron update rule
def update(w, X, y, i):
    x_i, y_i = X[i], y[i]
    if y_i * np.dot(w, x_i) < 1:
        w += y_i * x_i
    else:
        w += 0
    return w

# Empirical risk
def empirical_risk(w, X, Y):
    return np.mean(np.maximum(0, 1 - Y * (np.dot(X, w))))

# Correct predictions
def correct_predictions(w, X, Y):
    return np.mean(Y * (np.dot(X, w)) >= 1)

In [8]:
# Test the implementation
random_seed = 123
max_iter = 1000
eps = 1e-8

# Intialize the model
w = np.zeros(X.shape[1])
losses = []
rng = np.random.default_rng(random_seed)

for j in range(max_iter):
    losses.append(empirical_risk(w, X, Y))
    i = rng.integers(0, len(X))
    w = update(w, X, Y, i)
    if losses[-1] < eps:
        losses.append(empirical_risk(w, X, Y))
        break
        
assert losses[-1] < eps
assert correct_predictions(w, X, Y) == 1

print(f"Model is trained in {j+1} iterations.")

Model is trained in 184 iterations.


**Task 6 [1 point]:** Update the implemntation above to collect indices of misclassified points.

In [9]:
# Perceptron update rule with missclassified flag
def update(w, X, y, i):
    x_i, y_i = X[i], y[i]
    if y_i * np.dot(w, x_i) < 1:
        missclassified = True
        w += y_i * x_i
    else:
        missclassified = False
        w += 0
    return w, missclassified

In [10]:
# Test the implementation
random_seed = 123
max_iter = 1000
eps = 1e-8

# Intialize the model
w = np.zeros(X.shape[1])
losses = []
rng = np.random.default_rng(random_seed)
M = set()
samples = []

for j in range(max_iter):
    losses.append(empirical_risk(w, X, Y))
    i = rng.integers(0, len(X))
    samples.append(i)
    w, missclasified = update(w, X, Y, i)
    if missclasified:
        M.add(i)
    if losses[-1] < eps:
        losses.append(empirical_risk(w, X, Y))
        break
        
assert losses[-1] < eps
assert correct_predictions(w, X, Y) == 1

print(f"Model is trained in {j+1} iterations.")

Model is trained in 184 iterations.


**Task 7 [1 point]:** Assert that by removing all the points that does not belong to $M$, the final solutions remains the same if we preserve the sampling.


In [11]:
w_full_data = w.copy()

# Intialize the model
w = np.zeros(X.shape[1])
losses = []

for i in samples:
    losses.append(empirical_risk(w, X, Y))
    if i in M:
        w, _ = update(w, X, Y, i)
    if losses[-1] < eps:
        losses.append(empirical_risk(w, X, Y))
        break

assert np.linalg.norm(w - w_full_data) < 1e-10

print("Removing complement of M does not change the model.")

Removing complement of M does not change the model.
